## keras

### 全连接层

全连接层：神经网络中最常用到的，实现对神经网络里的神经元激活。

Dense（units, activation=’relu’, use_bias=True）

### 卷积层

卷积层：卷积操作分为一维、二维、三维，分别为Conv1D、Conv2D、Conv3D。一维卷积主要应用于以时间序列数据或文本数据，二维卷积通常应用于图像数据。由于这三种的使用和参数都基本相同，所以主要以处理图像数据的Conv2D进行说明

Conv2D(filters, kernel_size, strides=(1, 1), padding='valid')
- filters 卷积核的个数
- kernel_size filter的大小
- strdes：步长，二维中默认为(1, 1)，一维默认为1
- Padding：补“0”策略，'valid'指卷积后的大小与原来的大小可以不同，'same'则卷积后大小与原来大小一致。

### 池化层

池化层：与卷积层一样，最大统计量池化和平均统计量池也有三种，分别为MaxPooling1D、MaxPooling2D、MaxPooling3D、AveragePooling1D、AveragePooling2D、AveragePooli ng3D，由于使用和参数基本相同，所以主要以MaxPooling2D进行说明。MaxPooling(pool_size=(2,2), strides=None, padding=’valid’)


参数说明：

- pool_size：长度为2的整数tuple，表示在横向和纵向的下采样样子，一维则为纵向的下采样因子

- padding：和卷积层的padding一样

In [1]:
import pandas as pd

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


数据处理

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
#正则化 转变为[0...1] 加快运算速度  
X_train = train.iloc[:, 1:].values.astype('float32') / 255  # Normalization
y_train = train.iloc[:, :1].values.astype('int32')  # 1st column is 'label' for images


X_test = test.values.astype('float32') / 255  # Normalization

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)  # Reshaping
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
print(X_train.shape)
print(X_test.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [4]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes = 10)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=666)

cnn训练

In [5]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation = "softmax"))

In [6]:
model.compile(optimizer = "SGD" , loss = "categorical_crossentropy", metrics=["accuracy"])

In [7]:
model.fit(X_train, Y_train, batch_size=100, verbose=1,epochs=5,validation_data=(X_val, Y_val))

Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 318s 8ms/step - loss: 0.7572 - acc: 0.7848 - val_loss: 0.2783 - val_acc: 0.9212
Epoch 2/5
37800/37800 [==============================] - 314s 8ms/step - loss: 0.2746 - acc: 0.9159 - val_loss: 0.2002 - val_acc: 0.9429
Epoch 3/5
37800/37800 [==============================] - 304s 8ms/step - loss: 0.2027 - acc: 0.9385 - val_loss: 0.1639 - val_acc: 0.9538
Epoch 4/5
37800/37800 [==============================] - 307s 8ms/step - loss: 0.1653 - acc: 0.9498 - val_loss: 0.1616 - val_acc: 0.9548
Epoch 5/5
37800/37800 [==============================] - 308s 8ms/step - loss: 0.1443 - acc: 0.9560 - val_loss: 0.1175 - val_acc: 0.9662


In [8]:
# In summary() we can understand the architechture we build and 
# we can understand about the totol number of parameters 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               3211520   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total para

In [9]:
model.save('my_model.h5')

In [10]:
predict_testdata = model.predict(X_test)

In [11]:
predict_testdata

array([[5.4139496e-06, 1.9607957e-09, 9.9995410e-01, ..., 8.6206762e-07,
        1.1977712e-06, 4.2169890e-07],
       [9.9986339e-01, 5.9906458e-09, 2.1564065e-05, ..., 1.2698506e-05,
        8.2470542e-08, 3.8004330e-06],
       [4.2059773e-04, 1.0213978e-03, 1.2856643e-03, ..., 2.5948901e-03,
        5.7905842e-02, 8.2858104e-01],
       ...,
       [5.3705350e-07, 6.3552726e-08, 4.1089214e-07, ..., 4.4109836e-09,
        1.0934472e-06, 6.2224813e-06],
       [2.6562708e-04, 8.8085750e-07, 2.3065056e-06, ..., 7.2738319e-04,
        5.3178999e-05, 9.4447362e-01],
       [6.0952254e-05, 4.2899663e-07, 9.9954599e-01, ..., 4.8196745e-07,
        4.5874098e-05, 4.3854645e-05]], dtype=float32)

In [12]:
predict_classes_testdata = model.predict_classes(X_test)

In [13]:
predict_classes_testdata

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [14]:
df = pd.DataFrame(predict_classes_testdata)
df.index.name='ImageId'
df.index+=1
df.columns=['Label']
df.to_csv('cnn_results.csv', header=True)